In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from tensorflow.python.ops import rnn, rnn_cell
import random
import string

# Hyperparameters

In [2]:
input_size = 499
hidden_layer = 5

sampling_rate = 200
striding = 100
train_steps = 1000

# Onset-time data

In [3]:
def seizure_time_parser(onset_times,patient_number,data_set_number):
    temp = str(onset_times[patient_number-1][data_set_number])
    curr_onset_time = []
    while temp.find('/')!=-1:
        curr_onset_time.append(int(temp[0:temp.find('/')]))
        temp=temp[temp.find('/')+1:]
    curr_onset_time.append(int(float(temp)))
    curr_onset_time=np.array(curr_onset_time)
    return curr_onset_time

In [4]:
seizure_file = open('/home/maestroj/medical_analysis/project2/seizure_times.csv','r',newline='')
reader = csv.reader(seizure_file,delimiter=',')
onset_times_temp=[]

for row in reader:
    onset_times_temp.append(row)
    
onset_times = []
total_patient = len(onset_times_temp)
event_number = np.zeros(total_patient,dtype=np.int32)
#number of events of (i+1) patient = event_number[i]

for i in range(total_patient):
    event_number[i] = len(onset_times_temp[i])-1

for p in range(total_patient):
    temp = []
    for d in range(event_number[p]):
        temp.append(seizure_time_parser(onset_times_temp,p+1,d+1))
    onset_times.append(temp)

In [5]:
print("The number of datasets of patients")
print(event_number)
print("1st patient's seizure time for each data_set")
print(onset_times[0][1])
print("9th patient's seizure time for each data_set")
print(onset_times[8])

The number of datasets of patients
[ 7  7  6  3 10  1  7 13 16  8  8  5  3  6 31  8  2  4]
1st patient's seizure time for each data_set
[781]
9th patient's seizure time for each data_set
[array([200]), array([312]), array([624]), array([762]), array([752]), array([772]), array([884]), array([502]), array([ 94, 258]), array([836]), array([650]), array([636]), array([576]), array([656]), array([638]), array([706])]


In [6]:
def stft_load(patient_number, dataset_number):
    input_folder_path = '/home/maestroj/medical_analysis/stft_data/'
    input_data_name = '{}_{}.csv'.format(patient_number,dataset_number)
    input_data_path = os.path.join(input_folder_path,input_data_name)
    input_file = open(input_data_path,'r',newline='')
    reader = csv.reader(input_file, delimiter=',')
    print("{} is loaded".format(input_data_name))
    temp = []
    for row in reader:
        temp.append(row)
    try:
        temp = np.array(temp, dtype=np.float32)
    except: 
        for i in range(len(temp)):
            for j in range(len(temp[0])):
                temp[i][j]=abs(complex(temp[i][j].replace('i','j')))
        temp = np.array(temp, dtype=np.float32)
        temp = np.transpose(temp)
    return temp

In [7]:
def stft_normal(dataset_number):
    input_folder_path = '/home/maestroj/medical_analysis/stft_normal/'
    input_data_name = '{}.csv'.format(dataset_number)
    input_data_path = os.path.join(input_folder_path,input_data_name)
    input_file = open(input_data_path,'r',newline='')
    reader = csv.reader(input_file, delimiter=',')
    print("{} is loaded".format(input_data_name))
    temp = []
    for row in reader:
        temp.append(row)
    try:
        temp = np.array(temp, dtype=np.float32)
    except: 
        for i in range(len(temp)):
            for j in range(len(temp[0])):
                temp[i][j]=abs(complex(temp[i][j].replace('i','j')))
        temp = np.array(temp, dtype=np.float32)
    temp = np.transpose(temp)
    return temp

In [8]:
def time_axis_maker(array, sec_per_cell,init = 0):
    final = init + sec_per_cell*(len(array)-1)
    return np.linspace(init,final,len(array))

In [9]:
temp = stft_load(8,1)
print(temp.shape)

8_1.csv is loaded
(499, 32769)


# Training_set

In [10]:
input_data = []
for i in range(total_patient):
    for j in range(event_number[i]):
        try:
            temp = stft_load(i+1,j+1)
            for k in range(len(temp)):
                input_data.append(temp[k])
        except FileNotFoundError:
            continue;
input_data = np.array(input_data)
print(input_data.shape)

1_1.csv is loaded
1_2.csv is loaded
1_3.csv is loaded
1_4.csv is loaded
1_5.csv is loaded
1_6.csv is loaded
1_7.csv is loaded
2_1.csv is loaded
2_2.csv is loaded
2_3.csv is loaded
2_4.csv is loaded
2_5.csv is loaded
2_6.csv is loaded
2_7.csv is loaded
3_1.csv is loaded
3_5.csv is loaded
3_6.csv is loaded
4_1.csv is loaded
4_2.csv is loaded
4_3.csv is loaded
5_2.csv is loaded
5_3.csv is loaded
5_4.csv is loaded
6_1.csv is loaded
7_1.csv is loaded
7_2.csv is loaded
7_3.csv is loaded
7_4.csv is loaded
7_5.csv is loaded
7_6.csv is loaded
7_7.csv is loaded
8_1.csv is loaded
8_2.csv is loaded
8_3.csv is loaded
8_4.csv is loaded
8_5.csv is loaded
8_6.csv is loaded
8_7.csv is loaded
8_9.csv is loaded
8_10.csv is loaded
8_11.csv is loaded
8_12.csv is loaded
8_13.csv is loaded
9_3.csv is loaded
9_4.csv is loaded
9_5.csv is loaded
9_6.csv is loaded
9_7.csv is loaded
9_8.csv is loaded
9_10.csv is loaded
9_11.csv is loaded
9_12.csv is loaded
9_13.csv is loaded
9_14.csv is loaded
9_15.csv is loaded


# Neural network

In [16]:
X = tf.placeholder(tf.float32,[None,input_size])

w1 = tf.Variable(tf.random_uniform([input_size,hidden_layer], -0.01, 0.01))
b1 = tf.Variable(tf.random_uniform([1, hidden_layer], -0.01, 0.01))

w2 = tf.Variable(tf.random_uniform([hidden_layer,input_size], -0.01, 0.01))
b2 = tf.Variable(tf.random_uniform([1, input_size], -0.01, 0.01))

H = tf.matmul(X, w1) + b1
hypothesis = tf.matmul(H, w2) + b2

cost = tf.reduce_mean(tf.square(X-hypothesis))
optimizer = tf.train.AdamOptimizer(1e-3)
train = optimizer.minimize(cost)

Tensor("rnn/add:0", shape=(1000, 1), dtype=float32)
Tensor("rnn/add_1:0", shape=(1500, 1), dtype=float32)
Tensor("rnn/Reshape_2:0", shape=(1000, 1), dtype=float32)


In [20]:
sess = tf.Session()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
for step in range(train_steps+1):
    sess.run(train, feed_dict={X: input_data})
    if step%100==0:
        print("{}th run cost : {} ".format(step, sess.run(cost, feed_dict={X : input_data})))

In [ ]:
tf.add_to_collection("input", X)
tf.add_to_collection("hypo", H)
saver.save(sess, 'auto')